In [1]:
!pip install unidecode
!pip install LSTM nltk
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
import requests
import os
import pandas as pd
import re
import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Paso 1: Obtener y preprocesar el texto
nltk.download('punkt')
nltk.download('stopwords')

path = "pg4300.txt"
with open(path) as f:
    text = f.read()

# Preprocesar texto (simplificado, considerar limpiar encabezados y pies de página del sitio)
text = text.lower()  # Convertir a minúsculas
# Asumiendo que 'text' es tu texto completo
sentences = text.split('.')  # Esto divide el texto en frases basándose en el punto.

# Crear un DataFrame con las frases
df_sentences = pd.DataFrame(sentences, columns=['sentence'])

# Eliminar espacios en blanco y filtrar frases vacías
df_sentences['sentence'] = df_sentences['sentence'].str.strip()
df_sentences = df_sentences[df_sentences['sentence'] != '']
#longitud en caracteres
df_sentences['length'] = df_sentences['sentence'].str.len()
average_length = df_sentences['length'].mean()
median_length = df_sentences['length'].median()
print("Longitud promedio de las frases:", average_length)
print("Longitud mediana de las frases:", median_length)

# Eliminar caracteres especiales y números
text = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑ ]', '', text)
# Normalizar acentos
text = unidecode.unidecode(text)

# Tokenización
words = word_tokenize(text)

# Eliminar palabras de parada
filtered_words = [word for word in words if word not in stopwords.words('english')]

# Reconstruir el texto
cleaned_text = ' '.join(filtered_words)

chars = sorted(list(set(cleaned_text)))  # Listar caracteres únicos
char_to_int = dict((c, i) for i, c in enumerate(chars))  # Crear diccionario de caracteres a índices

# Crear secuencias de caracteres y sus próximos caracteres
maxlen = 50
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Número de secuencias:', len(sentences))

# Vectorizar las secuencias de entrada y etiquetas
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_int[char]] = 1
    y[i, char_to_int[next_chars[i]]] = 1

# Paso 2: Construir el modelo RNN
model = Sequential()
# Primera capa LSTM con return_sequences=True para apilar otra capa LSTM después
model.add(LSTM(128, return_sequences=True, input_shape=(maxlen, len(chars))))
# Opcionalmente, agrega Dropout para regularización
model.add(tf.keras.layers.Dropout(0.2))

# Segunda capa LSTM
model.add(LSTM(128))
# Otra capa de Dropout
model.add(tf.keras.layers.Dropout(0.2))

# Capa de salida
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='RMSprop')

# Paso 3: Entrenar el modelo
model.fit(x, y, batch_size=64, epochs=10)  # Ajusta el número de épocas según sea necesario
model.save('ulisses_0.2_token.keras')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.1 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Longitud promedio de las frases: 68.61963819470347
Longitud mediana de las frases: 36.0
Número de secuencias: 474818
Epoch 1/10
7420/7420 [==============================] - 63s 8ms/step - loss: 2.2588
Epoch 2/10
7420/7420 [==============================] - 60s 8ms/step - loss: 1.9801
Epoch 3/10
7420/7420 [==============================] - 58s 8ms/step - loss: 1.8959
Epoch 4/10
7420/7420 [==============================] - 58s 8ms/step - loss: 1.8467
Epoch 5/10
7420/7420 [==============================] - 60s 8ms/step - loss: 1.8137
Epoch 6/10
7420/7420 [==============================] - 57s 8ms/step - loss: 1.7877
Epoch 7/10
7420/7420 [==============================] - 58s 8ms/step - loss: 1.7671
Epoch 8/10
7420/7420 [==============================] - 57s 8ms/step - loss: 1.7505
Epoch 9/10
7420/7420 [==============================] - 57s 8ms/step - loss: 1.7380
Epoch 10/10
7420/7420 [==============================] - 57s 8ms/step - loss: 1.7267
